# Ethereum 2.0 Merge Proof-of-Concept

This notebook provides a proof-of-concept for the [Ethereum 2.0 Quick Merge Proposal](https://notes.ethereum.org/@vbuterin/B1mUf6DXO). A notable difference from the proposal is that Eth2 blocks in this PoC only contain Eth1 block hashes, instead of complete Eth1 blocks.

An unchanged Eth1 spec (from `py-evm`) has been used here.
The Eth2 spec has been slightly modified, and can be found in [this branch on `eth2.0-specs`](https://github.com/ethereum/eth2.0-specs/compare/adiasg-quick-merge-poc).

## Prerequisites
- Python3, Jupyter Notebook
- Run `install.sh` to:
    - download and install Eth1 & Eth2 python modules in a venv.
    - install the venv as a kernel (named `merge-spec-poc`) for Jupyter notebook.
- Choose the `merge-spec-poc` kernel to execute this Jupyter notebook.

## Other Merge Specification Efforts
- Mikhail Kalinin: https://github.com/ethereum/eth2.0-specs/pull/2257

In [16]:
from eth1 import Eth1Rpc
from eth.chains.base import MiningChain, Chain
from eth.vm.forks import IstanbulVM
import eth.tools.builder.chain as builder
        
# Initialize Eth1 chain builder
Eth1Chain = builder.build(MiningChain, builder.fork_at(IstanbulVM, 0))
Eth1Chain = builder.enable_pow_mining(Eth1Chain)
eth1_rpc = Eth1Rpc(Eth1Chain)

# Build a short Eth1 chain
eth1_blocks = [eth1_rpc.consensus_assembleBlock() for i in range(9)]
print_eth1_block(eth1_blocks[-1])

TypeError: print_eth1_block() missing 1 required positional argument: 'eth1_rpc'

In [3]:
import eth2spec.phase0.spec as spec

# Fill in the Eth1 RPC functions into the Eth2 state transition logic

def _is_valid_eth1_block(eth1_block_hash: spec.Bytes32) -> bool:
    assert eth1_rpc.is_accepted_block(eth1_block_hash)
    assert eth1_rpc.get_score(eth1_block_hash) >= spec.TRANSITION_TOTAL_DIFFICULTY
    return True

def _is_eth1_parent_block(parent_block_hash: spec.Bytes32, current_block_hash: spec.Bytes32) -> bool:
    assert eth1_rpc.is_parent_block(parent_block_hash, current_block_hash)
    return True

spec.is_valid_eth1_block = _is_valid_eth1_block
spec.is_eth1_parent_block = _is_eth1_parent_block

In [4]:
# Eth2 chain building tools
from eth2spec.test.helpers import genesis

genesis_state = genesis.create_genesis_state(spec, [32*10**9 for i in range(100)], 16*10**9)
genesis_block = spec.BeaconBlock(state_root=genesis_state.hash_tree_root())
store = spec.get_forkchoice_store(genesis_state, genesis_block)

In [5]:
from eth2spec.test.helpers.block import build_empty_block_for_next_slot, sign_block
from eth2spec.test.helpers.state import state_transition_and_sign_block, next_epoch

# Push the Eth2 state to some arbitraty slot
time = spec.SECONDS_PER_SLOT * 1000
spec.on_tick(store, time)
state = genesis_state.copy()
next_epoch(spec, state)

# Build a short Eth2 chain
for i in range(5):
    block = build_empty_block_for_next_slot(spec, state)
    block.body.application_block_hash = spec.ZERO_HASH
    signed_block = state_transition_and_sign_block(spec, state, block)

def print_eth2_state(state):
    print("Eth2 State:")
    print(f"\tSlot: {state.slot}")
    print(f"\tPrev. Eth1 Block: {state.previous_application_block_hash}")
    if state.previous_application_block_hash != spec.ZERO_HASH:
        print_eth1_block(eth1_rpc.chain.get_block_by_hash(state.previous_application_block_hash), "\t")

# This is the Eth2 state before the merge
print_eth2_state(state)

Eth2 State:
	Slot: 37
	Prev. Eth1 Block: 0x0000000000000000000000000000000000000000000000000000000000000000


In [6]:
# TRANSITION_TOTAL_DIFFICULTY is set to 10
# eth1_blocks[7] is the last PoW block
# eth1_blocks[8] is the first transition block

block = build_empty_block_for_next_slot(spec, state)
# Fill in the last PoW mined block
block.body.application_block_hash = eth1_blocks[8].hash
signed_block = state_transition_and_sign_block(spec, state, block)
print_eth2_state(state)

Eth2 State:
	Slot: 38
	Prev. Eth1 Block: 0x83e59fb2f65c04524e379d831dcf58bb9d835a0555d5d80dffb632adcac7615e
	Eth1 Block #9
		Parent Hash:	0xd9c04df336b521d5efe3141be789526fe89924946b84ace62bf1cfde6cbdbbe7
		Block Hash:	0x83e59fb2f65c04524e379d831dcf58bb9d835a0555d5d80dffb632adcac7615e
		Block Score:	10


## The Merge has happened!
## The last PoW block has been included in the Eth2 chain.
## All future Eth1 blocks will be produced by Eth2 PoS validator.

In [7]:
# Progress the Eth2 chain after the merge
for i in range(5):
    block = build_empty_block_for_next_slot(spec, state)
    # The eth1_rpc.mine_block() call in the next line is run by the Eth2 validator on its Eth1 node.
    # These blocks are not PoW intensive -- their difficulty will be 1.
    block.body.application_block_hash = eth1_rpc.mine_block().hash
    signed_block = state_transition_and_sign_block(spec, state, block)
    print_eth2_state(state)

Eth2 State:
	Slot: 39
	Prev. Eth1 Block: 0xaa5d56dd12d03563deabc79f80799d8dd6b64401d42315532b4d43a92ba2d97a
	Eth1 Block #10
		Parent Hash:	0x83e59fb2f65c04524e379d831dcf58bb9d835a0555d5d80dffb632adcac7615e
		Block Hash:	0xaa5d56dd12d03563deabc79f80799d8dd6b64401d42315532b4d43a92ba2d97a
		Block Score:	11
Eth2 State:
	Slot: 40
	Prev. Eth1 Block: 0x43f21c5c0c2bdfadb2f84c9fe8668cc6660abb372704289e0560d0aef7cafb1d
	Eth1 Block #11
		Parent Hash:	0xaa5d56dd12d03563deabc79f80799d8dd6b64401d42315532b4d43a92ba2d97a
		Block Hash:	0x43f21c5c0c2bdfadb2f84c9fe8668cc6660abb372704289e0560d0aef7cafb1d
		Block Score:	12
Eth2 State:
	Slot: 41
	Prev. Eth1 Block: 0x65b3f273f831c3baa5fb0c7bb13387b3be73fadf1c12c4da1f2740961786e9ab
	Eth1 Block #12
		Parent Hash:	0x43f21c5c0c2bdfadb2f84c9fe8668cc6660abb372704289e0560d0aef7cafb1d
		Block Hash:	0x65b3f273f831c3baa5fb0c7bb13387b3be73fadf1c12c4da1f2740961786e9ab
		Block Score:	13
Eth2 State:
	Slot: 42
	Prev. Eth1 Block: 0xedac74fd5c82566663008567c1d434ff32d2acf09b

In [8]:
def set_head(chain, block):
    chain.import_block(block)
    chain.headerdb._set_as_canonical_chain_head(chain.headerdb.db, block.header, GENESIS_PARENT_HASH)
    chain.header = chain.create_header_from_parent(block.header)
    return chain.get_block_by_header(chain.get_canonical_head())

c1, c2 = builder.build(
    eth1_rpc.chain,
    builder.chain_split(
        (builder.mine_block(extra_data=b'chain-c1'),),
        (builder.mine_block(extra_data=b'chain-c2'), builder.mine_block(extra_data=b'1'), builder.mine_block(extra_data=b'2'), builder.mine_block(extra_data=b'3')),
    )
)
b1 = c1.get_block_by_header(c1.get_canonical_head())
b2 = c2.get_block_by_header(c2.get_canonical_head())
print(b1)
print(b2)

Block #15-0x31bc..8082
Block #18-0x54ff..87c8


In [9]:
# print(c2.get_block_by_header(c2.get_canonical_head()))
print(c2.get_canonical_block_by_number(16))

Block #16-0x5aeb..92fe


In [10]:
set_head(c2, b1)

<IstanbulBlock(#Block #15-0x31bc..8082)>

In [11]:
print(c2.get_block_by_header(c2.get_canonical_head()))

Block #15-0x31bc..8082


In [12]:
c2 = builder.build(
    c2,
    builder.mine_block(extra_data=b'newblk')
)
block = c2.get_block_by_header(c2.get_canonical_head())
print(block)

Block #16-0x0271..2a52


In [13]:
block.header.hash

b'\x02q\xab\xf5\x96\xc3A\x83(\xf0W\x17\x89\x0b\xd4\x87\x89\xf26o\xabF\x12\x1c\x98|z\xc0\xe3\xef*R'